## Simplifiation (Summarization)
Use  [RuSimpleSentEval](https://github.com/DanAnastasyev/RuSimpleSentEval)

In [ ]:
!pip install transformers
!pip install SentencePiece 
!pip install razdel

     |████████████████████████████████| 2.3MB 8.0MB/s 
     |████████████████████████████████| 3.3MB 33.2MB/s 
     |████████████████████████████████| 901kB 30.3MB/s 
     |████████████████████████████████| 1.2MB 7.2MB/s 


In [ ]:
%%bash

download_from_google_drive() {
    wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate "https://docs.google.com/uc?export=download&id=$1" -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$1" -O $2 && rm -rf /tmp/cookies.txt
}

download_from_google_drive "1WfEq9Jfqi9sQQZXzDA3_MSHVmccbGgX-" simplifier.tar
tar -xvf simplifier.tar && rm simplifier.tar


simplifier/
simplifier/control_token_mapping.json
simplifier/pytorch_model.bin
simplifier/config.json


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
import json
from transformers import MBartForConditionalGeneration, MBartTokenizer

In [ ]:
#Base
import pandas as pd
import numpy as np
from tqdm import tqdm

#NLP
from razdel import sentenize, tokenize

In [ ]:
with open('simplifier/control_token_mapping.json') as f:
    control_token_mapping = json.load(f)

tokenizer = MBartTokenizer.from_pretrained('facebook/mbart-large-cc25')
model = MBartForConditionalGeneration.from_pretrained('simplifier')

In [ ]:

def _find_control_token_value(control_token_mapping, key_prefix, key_suffix, delta):
    value = control_token_mapping.get(f'{key_prefix}_{key_suffix - delta}')
    if value:
        return value
    value = control_token_mapping.get(f'{key_prefix}_{key_suffix + delta}')
    if value:
        return value
    return _find_control_token_value(control_token_mapping, key_prefix, key_suffix, delta + 0.05)


def _get_control_tokens(control_token_mapping, levenshtein_similarity, chars_fraction, word_rank):
    tokens = [
        _find_control_token_value(control_token_mapping, 'LevSim', levenshtein_similarity, 0.),
        _find_control_token_value(control_token_mapping, 'NbChars', chars_fraction, 0.),
        _find_control_token_value(control_token_mapping, 'WordRank', word_rank, 0.),
    ]

    return ' '.join(tokens)


def generate(model, tokenizer, control_token_mapping, text, levenshtein_similarity, chars_fraction, word_rank):
    control_tokens = _get_control_tokens(control_token_mapping, levenshtein_similarity, chars_fraction, word_rank)
    
    inputs = tokenizer(control_tokens + ' ' + text, return_tensors='pt')
    outputs = model.generate(**inputs, decoder_start_token_id=tokenizer.unk_token_id)[0]

    return tokenizer.decode(outputs, skip_special_tokens=True)




# Load data

In [ ]:
#Add google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/MyDrive/Diplom/Generation/s2t/"

In [ ]:
#open file
with open("/content/drive/MyDrive/Diplom/Generation/s2t/text.txt", encoding= 'cp1251') as txt:
  text = txt.read()
text

'"Семь навыков высокоэффективных людей" - это одна из самых популярных книг для развития личности, которая была переведена на 38 языков. Многомиллионные тиражи говорят о том, что среди литературы по саморазвитию она обладает непререкаемым авторитетом. Эта книга более 7 лет возглавляла список бестселлеров и по результатам опроса читателей журнала для руководителей была названа самой влиятельной книгой по бизнесу 20 века. Журнал Time назвал автора книги Доктора Стивена Кови одним из 25 самых влиятельных американцев. В данном видео вы узнаете ключевые тезисы этой уникальной книги. В начале книги Стивен Кови объясняет понятие навыка как пересечение знания, умения и желания. Знание это теоретическая база, определяющая, что делать и зачем. Умение определяет как делать, а желание - это мотивация, то есть хочу делать. Чтобы что-то в своей жизни превратить в навык, необходимы все три компонента.\n"Семь навыков" - это навыки эффективности. Базируясь на принципах, они приносят наилучшие из возмож

In [ ]:
df_bleu = pd.read_csv('https://raw.githubusercontent.com/Digital-Pushkin-Lab/RuAdapt/main/df_bleu.csv', sep= ',', error_bad_lines=False)
df_bleu.head()

,source,target,level
0,"— И очень хорошо сделали, что не обратились к ...","— И очень хорошо сделали, что не обратились к ...",b1_c1
1,"Чужестранец все прохаживался, но, увидев, что ...","Батюшки мои, думаю себе: вот чудак-то! И прика...",b1_c1
2,"Шипучин. Нет, я не вынесу! (Плачет.) Не вынесу...","Шипучин. Нет, я не вынесу! (Плачет.) Не вынесу...",a2_b1
3,"— Очень много, — краснея и с глупой, небрежной...","— Очень много, — краснея и с глупой улыбкой, к...",b1_c1
4,Вечер и вправду складывался весело.,"Вечер и, правда, становился всё более весёлым....",b1


In [ ]:

df_rsse = pd.read_csv('https://raw.githubusercontent.com/dialogue-evaluation/RuSimpleSentEval/main/dev_sents.csv', sep= ',',index_col=0, error_bad_lines=False)
df_rsse.head()

,INPUT:source,OUTPUT:output
3,14 декабря 1944 года рабочий посёлок Ички был ...,14 декабря 1944 года рабочий посёлок Ички пере...
4,1960 году была выпущена модель 172A. Изменени...,В 1960 году вышла модель 172А. Отличие в хвост...
5,1960 году была выпущена модель 172A. Изменени...,В выпущенной в 1960 году модель имела изменени...
6,1960 году была выпущена модель 172A. Изменени...,Изменения: в хвосте и руле направления с обрат...
7,1960 году была выпущена модель 172A. Изменени...,"Модель 172А с другим хвостовым оперением, кре..."


In [ ]:
levenshtein_similarity = 0.9 
chars_fraction = 0.4
word_rank = 3.2

In [ ]:

text_simple = ""
# for s in sentenize(text):
#   print(s.text, ' vs ', 
#         generate(model, tokenizer, control_token_mapping, s.text, levenshtein_similarity, chars_fraction, word_rank)
#         ,'\n')
text_simple = '\n'.join([generate(model, tokenizer, control_token_mapping, s.text
                                  , levenshtein_similarity, chars_fraction, word_rank)\
                         for s in tqdm(sentenize(text))])

text_simple

In [ ]:
nn = 50

subsample = np.random.choice(df_bleu.index, nn, replace=False)
bleu_text = df_bleu.loc[subsample].apply(lambda x: ' '.join(x),axis = 0)

len([s for s in sentenize(bleu_text[0])])

166

In [ ]:
bleu_text.to_csv(path + 'sample.csv')

In [ ]:
text_simple_bleu = ' '.join([generate(model, tokenizer, control_token_mapping, s.text
                          , levenshtein_similarity, chars_fraction, word_rank)\
                  for s in tqdm(sentenize(bleu_text[0]))])

In [ ]:
text_simple_bleu

'Оставшись один на перекрестке, князь осмотрелся кругом, быстро перешел через улицу, близко подошел к освещенному окну одной дачи. Я решилась говорить с вами об одном деле. И объясните мне леди Эстер? Почему именно ей завещано? Какая связь? Вот что выяснить Вы купили? Хамите. На другой день, в 11 часов утра, Вронский выехал на станцию Петербургской железной дороги. Он не ошибся. Не успели еще замолкнуть раскаты и переливы председательского голоса. Они несли. Один держал его за руки. С тем, кто не испугался бы. Что-нибудь? Закажи порцию селянки Я подумаю Узнаю! - воскликнул Вася. Узнаю Колю Шипучин. Позвольте Берет от нее прошение. Боцмана засвистали. Все матросы были на палубе. У вас очень много шипящих букв. У нас язык легкий. А у вас свистят. Вершинин. О чем? Давайте помечтаем о той жизни, какая будет после нас. Балаганов, - Шура Балаганов. Но и в Западном отряде готовились основательно. Штурм был назначен на сегодня, и все говорили. Эраст Петрович начертил для Вари прутиком все туре

In [ ]:
with open('drive/MyDrive/Diplom/Generation/s2t/summary.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write(text_simple_bleu)

[RuSimpleSentEval](https://github.com/dialogue-evaluation/RuSimpleSentEval/blob/main/dev_sents.csv)

In [ ]:
# nn = 50

# subsample = np.random.choice(df_rsse.index, nn, replace=False)
rsse_text = df_rsse.apply(lambda x: ' '.join(x),axis = 0)

len([s for s in sentenize(rsse_text[0])])

3397

In [ ]:
text_simple_rsse = ' '.join([generate(model, tokenizer, control_token_mapping, s.text
                          , levenshtein_similarity, chars_fraction, word_rank)\
                  for s in tqdm(sentenize(rsse_text[0]))])

3397it [8:50:05,  9.36s/it]


In [ ]:
with open('drive/MyDrive/Diplom/Generation/s2t/summary rsse.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write(text_simple_rsse)